# Prétraitement des données de tweets

**Auteur :** Mehdi MUNIM

**Date :** 2023-11-26

**Description :**

Ce notebook prétraite les données de tweets en utilisant les fonctions définies dans le fichier `preprocessing.py`.

In [1]:
#@title 1. Importation des librairies

import pandas as pd
import sys

# Ajouter le chemin du dossier src pour importer les modules
sys.path.append('../src')
from preprocessing import *

## 2. Chargement des données

In [2]:
#@title 2.1 Chargement des données

# Charger les données du fichier CSV
df = pd.read_csv('../data/raw/training.1600000.processed.noemoticon.csv', encoding='latin-1', header=None)

# Renommer les colonnes
df.columns = ['sentiment', 'id', 'date', 'query', 'user', 'text']

# Afficher les premières lignes du DataFrame
print("Premières lignes du DataFrame :")
display(df.head())

Premières lignes du DataFrame :


,sentiment,id,date,query,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [3]:
#@title Échantillonage des données

# Mélanger les données
df = df.sample(frac=1, random_state=42)

# Échantillonner 10 % des données
df = df.sample(frac=0.1, random_state=42)

## 3. Prétraitement des données

In [4]:
#@title 3.1 Nettoyage du texte

# Appliquer la fonction de nettoyage du module preprocessing
df['text'] = df['text'].map(lambda tweet: clean_tweet(tweet))

# Afficher les premières lignes du DataFrame avec les tweets nettoyés
print("Premières lignes du DataFrame avec les tweets nettoyés :")
display(df.head())

Premières lignes du DataFrame avec les tweets nettoyés :


,sentiment,id,date,query,user,text
480874,0,2179423207,Mon Jun 15 09:05:02 PDT 2009,NO_QUERY,fashionNERD,internet slow today like
1352277,4,2046335842,Fri Jun 05 12:01:59 PDT 2009,NO_QUERY,SLmoore,hammer everything look like nail
409163,0,2059580298,Sat Jun 06 17:13:08 PDT 2009,NO_QUERY,Ashleyiswhatsup,kid crazy save
796992,0,2327988448,Thu Jun 25 09:02:03 PDT 2009,NO_QUERY,peachtweet,puppy libby scratched heel bleeding got bandaid
814154,4,1550733689,Sat Apr 18 07:05:59 PDT 2009,NO_QUERY,mchollee,okay ipod synced finally happy anywho painting...


In [5]:
#@title 3.2 Tokenization

from transformers import BertTokenizer

# Charger le tokenizer BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokeniser les tweets
X = tokenizer(
    df['text'].tolist(),
    padding=True,
    truncation=True,
    max_length=128,  # Spécifier la longueur maximale des séquences
    return_tensors="tf"
)


2024-11-26 17:58:41.269371: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-26 17:58:44.660348: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 35200000 exceeds 10% of free system memory.
2024-11-26 17:58:44.860305: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 35200000 exceeds 10% of free system memory.
2024-11-26 17:58:44.982769: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 35200000 exceeds 10% of free system memory.


In [6]:
#@title Conversion des labels en 0 et 1

df['sentiment'] = df['sentiment'].map({0: 0, 4: 1})  # Convertir les labels en 0 et 1

In [7]:
#@title 3.3 Vectorisation

from sklearn.model_selection import train_test_split

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['sentiment'], test_size=0.2, random_state=42)

# Vectorisation TF-IDF
X_train_tfidf, X_test_tfidf = vectorize_tfidf(X_train, X_test)

# Vectorisation Word2Vec
X_train_w2v, X_test_w2v = vectorize_word2vec(X_train, X_test)

# Vectorisation Doc2Vec
X_train_d2v, X_test_d2v = vectorize_doc2vec(X_train, X_test)

In [8]:
#@title 3.4 Réduction de dimensionnalité

# Réduction de dimensionnalité pour TF-IDF
X_train_tfidf, X_test_tfidf = reduce_dimensionality(X_train_tfidf, X_test_tfidf, y_train, k=1000)

# Réduction de dimensionnalité pour Word2Vec
X_train_w2v, X_test_w2v = reduce_dimensionality(X_train_w2v, X_test_w2v, y_train, k=100)

# Réduction de dimensionnalité pour Doc2Vec
X_train_d2v, X_test_d2v = reduce_dimensionality(X_train_d2v, X_test_d2v, y_train, k=100)


## 4. Sauvegarde des données prétraitées

In [9]:
#@title 4.1 Sauvegarde des données

import os
import pickle 

# Chemin d'accès au dossier data
data_path = '../data/processed/'

# Créer le dossier s'il n'existe pas
os.makedirs(data_path, exist_ok=True)

# Sauvegarde des données pour chaque méthode
def save_data(X_train, X_test, methode):
    with open(data_path + f'X_train_{methode}.pickle', 'wb') as f:
        pickle.dump(X_train, f)
    with open(data_path + f'X_test_{methode}.pickle', 'wb') as f:
        pickle.dump(X_test, f)

save_data(X_train_tfidf, X_test_tfidf, "tfidf")
save_data(X_train_w2v, X_test_w2v, "word2vec")
save_data(X_train_d2v, X_test_d2v, "doc2vec")

# Sauvegarde de y_train et y_test
with open(data_path + 'y_train.pickle', 'wb') as f:
    pickle.dump(y_train, f)

with open(data_path + 'y_test.pickle', 'wb') as f:
    pickle.dump(y_test, f)